In [ ]:
# default_exp notify
# hide
_FNAME='notify'

import unittest
from unittest import mock
from nbdev.export import notebook2script
import os
TESTCASE = unittest.TestCase()
_nbpath = os.path.join(_dh[0], _FNAME+'.ipynb')
notebook2script(_nbpath)

Converted notify.ipynb.


# Notify

In [ ]:
#export
import datetime
import logging
logger = logging.getLogger()

from sewing.mailer import send
from sewing.db import get_guild_emails
import sewing.display

In [ ]:
#export 

def yesterday():
    now = datetime.datetime.now()
    return now - datetime.timedelta(days=1)
    

class Retriever:
    def __init__(self, after=None):
        self.after = after

    async def get_all_messages_in_channel(self, chan):
        '''
        Returns all messages in all open threads in the channel
        That were sent since `after`
        '''
        messages = {}
        all_threads = [] if chan.threads is None else chan.threads
        for thread in all_threads:
            logger.info("THREAD {}".format(thread))
            messages[thread] = await self.get_all_messages_in_thread(thread)
        return messages

    async def get_all_messages_in_thread(self, thread):
            messages = await self.retrieve(thread)
            for msg in messages:
                logger.info("{}".format(msg))
            return messages

    async def retrieve(self, messagable, after=None, **kwargs):
        '''
        messagable = thread or channel
        See other kwargs
        https://discordpy.readthedocs.io/en/stable/api.html#discord.TextChannel.history
        '''
        after = after or self.after or yesterday()
        messages = await messagable.history(after=after, **kwargs).flatten()
        return messages


In [ ]:
#export

def send_content(content:dict, guild:discord.Guild):
    to_emails = get_guild_emails(guild)
    html_content = sewing.display.html_content(content)
    subject = '[{}] Daily Thread Summary'.format(guild.name)
    return send(to_emails, subject, html_content)    